In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('Boston.csv')
df = df.fillna(df.mean())
df = df.drop([76], axis = 0)

t = df[['PRICE']]
x = df.loc[:, ['RM', 'PTRATIO', 'LSTAT']]

sc = StandardScaler()
sc_x = sc.fit_transform(x)
sc2 = StandardScaler()
sc_t = sc2.fit_transform(t)

/var/folders/pl/l83vr0j523j0hp3dngvsrhqc0000gn/T/ipykernel_68176/3701213424.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


In [7]:
from sklearn.preprocessing import PolynomialFeatures

#累乗列と交互作用特徴量を一括追加する
pf = PolynomialFeatures(degree = 2, include_bias = False)
pf_x = pf.fit_transform(sc_x)
pf_x.shape

(99, 9)

In [9]:
from sklearn.linear_model import LinearRegression
# 過学習が起きることを確認
x_train, x_test, y_train, y_test = train_test_split(pf_x, sc_t, test_size = 0.3, random_state = 0)
model = LinearRegression()
model.fit(x_train, y_train)

print(model.score(x_train, y_train))
model.score(x_test, y_test)

0.8710525685992707


0.7854929935582584

In [10]:
from sklearn.linear_model import Ridge
#リッジ回帰でモデルを作成
ridgeModel = Ridge(alpha = 10)
ridgeModel.fit(x_train, y_train)
print(ridgeModel.score(x_train, y_train))
print(ridgeModel.score(x_test, y_test))

0.8607320524729508
0.8458730019328173


In [12]:
# 正規化項の定数を0.01~20まで0.01刻みで検証するコード
maxScore = 0
maxIndex = 0

for i in range(1, 2001):
    num = i/100
    ridgeModel = Ridge(random_state = 0, alpha = num)
    ridgeModel.fit(x_train, y_train)
    
    result = ridgeModel.score(x_test, y_test)
    if result > maxScore:
        maxScore = result
        maxIndex = num
    
print(maxIndex, maxScore)

17.62 0.8528754801497631


In [15]:
# 線形回帰の係数(絶対値)の合計
print(sum(abs(model.coef_)[0]))
# リッジ回帰の合計
print(sum(abs(ridgeModel.coef_)[0]))

1.5566745983288375
1.2152800824026813


In [17]:
from sklearn.linear_model import Lasso

x_train, x_test, y_train, y_test = train_test_split(pf_x, sc_t, test_size = 0.3, random_state = 0)

# ラッソ回帰のモデル作成
model = Lasso(alpha = 0.1)
model.fit(x_train, y_train)

print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

0.8224680202036665
0.858846785318774


In [18]:
weight = model.coef_ # 係数抜き出す

pd.Series(weight, index = pf.get_feature_names())

AttributeError: 'PolynomialFeatures' object has no attribute 'get_feature_names'

In [24]:
df2 = pd.read_csv('Boston.csv')
df2 = df2.fillna(df.mean())

t = df2[['PRICE']]
x = df2.loc[:, 'ZN':'LSTAT']

x_train, x_test, y_train, y_test = train_test_split(x, t, test_size = 0.3, random_state = 0)

/var/folders/pl/l83vr0j523j0hp3dngvsrhqc0000gn/T/ipykernel_68176/3148534953.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df2 = df2.fillna(df.mean())


In [25]:
# 回帰木
from sklearn.tree import DecisionTreeRegressor

# 木の深さの最大を10と設定
model = DecisionTreeRegressor(max_depth = 10, random_state = 0)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.59433275545417

In [26]:
pd.Series(model.feature_importances_, index = x.columns)

ZN         0.000252
INDUS      0.007301
CHAS       0.000000
NOX        0.001967
RM         0.759547
AGE        0.139388
DIS        0.013635
RAD        0.000404
TAX        0.013975
PTRATIO    0.001913
B          0.003331
LSTAT      0.058287
dtype: float64